In [1]:
from dash import Dash, dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os
from IPython.display import clear_output

# ✅ Debug: Print current working directory and paths
print("Current working directory:", os.getcwd())
print("Checking furniture directory paths...")

# Try multiple possible paths
possible_paths = [
    os.path.abspath("../assets/object/furniture"),
    os.path.abspath("assets/object/furniture"),
    os.path.abspath("./assets/object/furniture"),
    os.path.join(os.getcwd(), "assets/object/furniture"),
    os.path.join(os.getcwd(), "../assets/object/furniture")
]

FURNITURE_DIR = None
for path in possible_paths:
    print(f"Trying path: {path}")
    if os.path.exists(path):
        FURNITURE_DIR = path
        print(f"✅ Found furniture directory at: {path}")
        break
    else:
        print(f"❌ Path does not exist: {path}")

if not FURNITURE_DIR:
    print("❌ Could not find furniture directory!")
    FURNITURE_ITEMS = []
    furniture_images = {}
else:
    # 📦 List furniture categories
    FURNITURE_ITEMS = sorted([
        item for item in os.listdir(FURNITURE_DIR)
        if os.path.isdir(os.path.join(FURNITURE_DIR, item))
    ])
    print(f"Found furniture items: {FURNITURE_ITEMS}")

# Try multiple asset folder configurations
asset_folders = [
    os.path.abspath("../assets"),
    os.path.abspath("assets"),
    os.path.abspath("./assets"),
    os.path.join(os.getcwd(), "assets"),
    os.path.join(os.getcwd(), "../assets")
]

ASSETS_FOLDER = None
for folder in asset_folders:
    if os.path.exists(folder):
        ASSETS_FOLDER = folder
        print(f"✅ Using assets folder: {folder}")
        break

# ✅ Create Dash app with multiple fallback configurations
app = JupyterDash(
    __name__,
    assets_folder=ASSETS_FOLDER if ASSETS_FOLDER else None,
    suppress_callback_exceptions=True
)
app.title = "🪑 Furniture Carousel"

# 📸 Build image dictionary with multiple path formats
furniture_images = {}
if FURNITURE_DIR and FURNITURE_ITEMS:
    for item in FURNITURE_ITEMS:
        item_path = os.path.join(FURNITURE_DIR, item)
        if os.path.exists(item_path):
            images = []
            for img in os.listdir(item_path):
                if img.lower().endswith((".jpg", ".jpeg", ".png", ".gif", ".bmp", ".webp")):
                    # Store the full absolute path for now
                    full_path = os.path.join(item_path, img)
                    images.append(full_path)
            furniture_images[item] = sorted(images)
            print(f"{item}: {len(images)} images found")

# 🎨 Layout
app.layout = html.Div([
    html.H2("🪑 Furniture Carousel", style={
        "textAlign": "center", "marginBottom": "25px", "marginTop": "10px",
        "fontSize": "28px", "fontWeight": "bold", "fontFamily": "Georgia, serif"
    }),
    
    html.Div([
        html.Label("Select furniture item:", style={
            "fontWeight": "normal", "display": "block", "marginBottom": "6px",
            "fontSize": "15px", "fontFamily": "Georgia, serif"
        }),
        dcc.Dropdown(
            id='item-dropdown',
            options=[{"label": item.replace("_", " ").capitalize(), "value": item} for item in FURNITURE_ITEMS],
            value=FURNITURE_ITEMS[0] if FURNITURE_ITEMS else None,
            clearable=False,
            style={"width": "300px", "margin": "0 auto"}
        )
    ], style={"textAlign": "center", "marginBottom": "20px"}),
    
    html.Div(id='carousel-container', style={"textAlign": "center"}),
    

])

# 🔁 Display carousel
@app.callback(
    Output('carousel-container', 'children'),
    Input('item-dropdown', 'value')
)
def update_carousel(item):
    if not item:
        return html.P("Please select a furniture item.", style={"textAlign": "center"})
    
    images = furniture_images.get(item, [])
    
    if not images:
        return html.P(f"No images available for {item}.", style={"textAlign": "center"})
    
    # Convert absolute path to relative path for web serving
    image_urls = []
    for img_path in images:
        # Try multiple URL formats
        relative_path = os.path.relpath(img_path, os.getcwd())
        web_path = relative_path.replace("\\", "/")  # Convert Windows paths
        
        # Remove leading ../ if present and try different formats
        url_attempts = [
            f"/{web_path}",
            web_path,
            f"/assets/{web_path.split('assets/')[-1]}" if 'assets/' in web_path else None,
            f"assets/{web_path.split('assets/')[-1]}" if 'assets/' in web_path else None
        ]
        
        # Use the first non-None attempt
        img_url = next((url for url in url_attempts if url), web_path)
        image_urls.append(img_url)
    
    filename = os.path.basename(images[0])
    
    return html.Div([
        html.Div([
            html.Button("⬅️", id="prev-button", n_clicks=0,
                        style={"fontSize": "24px", "padding": "10px 15px", "marginRight": "20px"}),
            html.Img(id="carousel-image", 
                     src=image_urls[0],
                     style={"height": "450px", "display": "inline-block", "verticalAlign": "middle"}),
            html.Button("➡️", id="next-button", n_clicks=0,
                        style={"fontSize": "24px", "padding": "10px 15px", "marginLeft": "20px"}),
        ], style={"textAlign": "center", "marginBottom": "20px"}),
        
        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=image_urls),
        html.Div(f"1/{len(images)} - {filename}", id='filename-display',
                 style={"textAlign": "center", "fontSize": "14px", "fontStyle": "italic", "fontFamily": "Georgia, serif"})
    ])

# 🔁 Navigation arrows
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images"
    
    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)
    
    # Get the original filename from the path
    original_paths = list(furniture_images.values())
    if original_paths:
        filename = os.path.basename(original_paths[0][index] if len(original_paths[0]) > index else "unknown")
    else:
        filename = "unknown"
    
    return images[index], index, f"{index + 1}/{len(images)} - {filename}"

# 🚀 Launch
clear_output(wait=True)
app.run(port=8125)